<a href="https://colab.research.google.com/github/MudassirABBASSi/Recommendation-System-/blob/main/Netflix_Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [420]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sna

In [421]:
# Create the Kaggle directory and copy your kaggle.json file there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Set permissions for the kaggle.json file (optional, but recommended)
!chmod 600 ~/.kaggle/kaggle.json

# Download "The Movies Dataset" from Kaggle using its dataset identifier
!kaggle datasets download -d rounakbanik/the-movies-dataset
!kaggle datasets download -d harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows
# Extract the downloaded ZIP file into the /content directory
import zipfile

zip_ref = zipfile.ZipFile('/content/imdb-dataset-of-top-1000-movies-and-tv-shows.zip','r')
zip_ref.extractall('/content')
zip_ref.close()



cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
License(s): CC0-1.0
the-movies-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows
License(s): CC0-1.0
imdb-dataset-of-top-1000-movies-and-tv-shows.zip: Skipping, found more recently modified local copy (use --force to force download)


In [422]:
df = pd.read_csv('/content/imdb_top_1000.csv')
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [423]:
df.columns

Index(['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director',
       'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')

In [424]:
# prompt: df['Released_Year'].value_counts().sns.barplot

# import seaborn as sns
# plt.figure(figsize=(20,18))
# sns.barplot(df['Released_Year'].value_counts(),orient='h')
# plt.show()


In [425]:
data = df[['Series_Title','Director','Genre', 'Star1', 'Star2', 'Star3', 'Star4']]


In [426]:
data

,Series_Title,Director,Genre,Star1,Star2,Star3,Star4
0,The Shawshank Redemption,Frank Darabont,Drama,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler
1,The Godfather,Francis Ford Coppola,"Crime, Drama",Marlon Brando,Al Pacino,James Caan,Diane Keaton
2,The Dark Knight,Christopher Nolan,"Action, Crime, Drama",Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
3,The Godfather: Part II,Francis Ford Coppola,"Crime, Drama",Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton
4,12 Angry Men,Sidney Lumet,"Crime, Drama",Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler
...,...,...,...,...,...,...,...
995,Breakfast at Tiffany's,Blake Edwards,"Comedy, Drama, Romance",Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen
996,Giant,George Stevens,"Drama, Western",Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker
997,From Here to Eternity,Fred Zinnemann,"Drama, Romance, War",Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed
998,Lifeboat,Alfred Hitchcock,"Drama, War",Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix


In [427]:
data.shape

(1000, 7)

In [428]:
df.isna().sum()

,0
Poster_Link,0
Series_Title,0
Released_Year,0
Certificate,101
Runtime,0
Genre,0
IMDB_Rating,0
Overview,0
Meta_score,157
Director,0


In [429]:
data['Genre'] = data['Genre'].str.replace(',',' ')

<ipython-input-429-1c4395064ef7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Genre'] = data['Genre'].str.replace(',',' ')


In [430]:
data

,Series_Title,Director,Genre,Star1,Star2,Star3,Star4
0,The Shawshank Redemption,Frank Darabont,Drama,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler
1,The Godfather,Francis Ford Coppola,Crime Drama,Marlon Brando,Al Pacino,James Caan,Diane Keaton
2,The Dark Knight,Christopher Nolan,Action Crime Drama,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
3,The Godfather: Part II,Francis Ford Coppola,Crime Drama,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton
4,12 Angry Men,Sidney Lumet,Crime Drama,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler
...,...,...,...,...,...,...,...
995,Breakfast at Tiffany's,Blake Edwards,Comedy Drama Romance,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen
996,Giant,George Stevens,Drama Western,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker
997,From Here to Eternity,Fred Zinnemann,Drama Romance War,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed
998,Lifeboat,Alfred Hitchcock,Drama War,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix


In [431]:
df['Series_Title'][4]

'12 Angry Men'

In [432]:
data.to_csv('data.csv',index=False)

# Part 2 preparation

In [433]:
# Create the Kaggle directory and copy your kaggle.json file there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Set permissions for the kaggle.json file (optional, but recommended)
!chmod 600 ~/.kaggle/kaggle.json

# Download "The Movies Dataset" from Kaggle using its dataset identifier
!kaggle datasets download -d rounakbanik/the-movies-dataset

# Extract the downloaded ZIP file into the /content directory
import zipfile

zip_ref = zipfile.ZipFile('/content/the-movies-dataset.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
License(s): CC0-1.0
the-movies-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


**we will use a the dataset**
#Credit.csv
#movies_metadata.csv

In [434]:
import pandas as pd
import numpy as np

movie  = pd.read_csv('/content/movies_metadata.csv')
credit = pd.read_csv('/content/credits.csv')

<ipython-input-434-e92641554436>:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie  = pd.read_csv('/content/movies_metadata.csv')


In [435]:
movie.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [436]:
movie['release_date']  = pd.to_datetime(movie['release_date'],errors='coerce')

In [437]:
movie['year'] = movie['release_date'].dt.year
movie['year'].value_counts().sort_index()

,count
year,
1874.0,1
1878.0,1
1883.0,1
1887.0,1
1888.0,2
...,...
2015.0,1905
2016.0,1604
2017.0,532


In [438]:
new_movie = movie.loc[movie['year'] == 2017,['title','genres','id','year']]
new_movie.head()

,title,genres,id,year
26560,Pirates of the Caribbean: Dead Men Tell No Tales,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",166426,2017.0
26561,Justice League,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",141052,2017.0
26565,Thor: Ragnarok,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",284053,2017.0
26566,Guardians of the Galaxy Vol. 2,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",283995,2017.0
30536,The King's Daughter,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",245842,2017.0


In [439]:
new_movie['id'] = new_movie['id'].astype(int)

In [440]:
data_p2 = pd.merge(new_movie,credit,on = 'id')

In [441]:
data_p2.head()

,title,genres,id,year,cast,crew
0,Pirates of the Caribbean: Dead Men Tell No Tales,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",166426,2017.0,"[{'cast_id': 1, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4c9cc3a36847f8236a65', 'de..."
1,Justice League,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",141052,2017.0,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '55ef66dbc3a3686f1700a52d', 'de..."
2,Thor: Ragnarok,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",284053,2017.0,"[{'cast_id': 0, 'character': 'Thor Odinson', '...","[{'credit_id': '56a93fa4c3a36872db001e7a', 'de..."
3,Guardians of the Galaxy Vol. 2,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",283995,2017.0,"[{'cast_id': 3, 'character': 'Peter Quill / St...","[{'credit_id': '59171547925141583c0315a6', 'de..."
4,The King's Daughter,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",245842,2017.0,"[{'cast_id': 0, 'character': 'King Louis XIV',...","[{'credit_id': '5431de49c3a36825d300007e', 'de..."


In [442]:
data_p2['Series_Title'] =data_p2['title']
data_p2.head()

,title,genres,id,year,cast,crew,Series_Title
0,Pirates of the Caribbean: Dead Men Tell No Tales,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",166426,2017.0,"[{'cast_id': 1, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4c9cc3a36847f8236a65', 'de...",Pirates of the Caribbean: Dead Men Tell No Tales
1,Justice League,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",141052,2017.0,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '55ef66dbc3a3686f1700a52d', 'de...",Justice League
2,Thor: Ragnarok,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",284053,2017.0,"[{'cast_id': 0, 'character': 'Thor Odinson', '...","[{'credit_id': '56a93fa4c3a36872db001e7a', 'de...",Thor: Ragnarok
3,Guardians of the Galaxy Vol. 2,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",283995,2017.0,"[{'cast_id': 3, 'character': 'Peter Quill / St...","[{'credit_id': '59171547925141583c0315a6', 'de...",Guardians of the Galaxy Vol. 2
4,The King's Daughter,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",245842,2017.0,"[{'cast_id': 0, 'character': 'King Louis XIV',...","[{'credit_id': '5431de49c3a36825d300007e', 'de...",The King's Daughter


In [443]:
import ast
data_p2['genres'] = data_p2['genres'].map(lambda x: ast.literal_eval(x))
data_p2['cast'] = data_p2['cast'].map(lambda x: ast.literal_eval(x))
data_p2['crew'] = data_p2['crew'].map(lambda x: ast.literal_eval(x))

In [444]:
def make_genreslist(x):
  gen = []
  st = " "
  for i in x:
    if i.get('name') == 'Science Fiction':
      scifi = 'Sci-Fi'
      gen.append(scifi)
    else:
      gen.append(i.get('name'))
  if gen == []:
      np.NAN
  else:
     return(st.join(gen))


data_p2['Genre'] = data_p2['genres'].map(lambda x: make_genreslist(x))
data_p2['Genre']

,Genre
0,Adventure Action Fantasy Comedy
1,Action Adventure Fantasy Sci-Fi
2,Action Adventure Fantasy Sci-Fi
3,Action Adventure Comedy Sci-Fi
4,Fantasy Action Adventure
...,...
526,Romance Comedy
527,Crime Comedy Action Family
528,Family Animation Romance Comedy
529,Crime Drama Thriller


In [445]:
def get_actor1(x):
  cast = []
  for i in x:
    cast.append(i.get('name'))
  if cast == []:
    return np.NAN
  else:
     return(cast[0])

data_p2['Star1'] = data_p2['cast'].map(lambda x: get_actor1(x))
data_p2['Star1']

,Star1
0,Johnny Depp
1,Ben Affleck
2,Chris Hemsworth
3,Chris Pratt
4,Pierce Brosnan
...,...
526,Inka Haapamäki
527,Lou Diamond Phillips
528,NaN
529,Sridevi Kapoor


In [446]:
def get_actor2(x):
  cast = []
  for i in x:
    cast.append(i.get('name'))
  if cast == []or len(cast)<=1:
    return np.NAN
  else:
     return(cast[1])

data_p2['Star2'] = data_p2['cast'].map(lambda x: get_actor2(x))
data_p2['Star2']

,Star2
0,Javier Bardem
1,Henry Cavill
2,Tom Hiddleston
3,Zoe Saldana
4,William Hurt
...,...
526,Rosa Honkonen
527,Wallace Shawn
528,NaN
529,Sajal Ali


In [447]:
def get_actor3(x):
  cast = []
  for i in x:
    cast.append(i.get('name'))
  if cast == []or len(cast)<=2:
    return np.NAN
  else:
    return(cast[2])

data_p2['Star3'] = data_p2['cast'].map(lambda x: get_actor3(x))
data_p2['Star3']

,Star3
0,Geoffrey Rush
1,Gal Gadot
2,Cate Blanchett
3,Dave Bautista
4,Benjamin Walker
...,...
526,Tiitus Rantala
527,Gina Holden
528,NaN
529,Akshaye Khanna


In [448]:
def get_actor4(x):
  cast = []
  for i in x:
    cast.append(i.get('name'))
  if cast == []or len(cast)<=3:
    return np.NAN
  else:
    return(cast[3])

data_p2['Star4'] = data_p2['cast'].map(lambda x: get_actor4(x))
data_p2['Star4']

,Star4
0,Brenton Thwaites
1,杰森·莫玛
2,Idris Elba
3,Vin Diesel
4,Kaya Scodelario
...,...
526,Santeri Helinheimo Mäntylä
527,Lulu Wilson
528,NaN
529,Nawazuddin Siddiqui


In [449]:
def get_director(x):
  dt = []
  st = " "
  for i in x:
    if i.get('job') == 'Director': #Fixed typo here
      dt.append(i.get('name'))
  if dt == []:
    return np.NAN
  else:
    return(st.join(dt))

data_p2['Director'] = data_p2['crew'].map(lambda x:get_director(x)    )
data_p2['Director']

,Director
0,Joachim Rønning Espen Sandberg
1,Zack Snyder
2,Taika Waititi
3,James Gunn
4,Sean McNamara
...,...
526,Hannaleena Hauru
527,Jonathan A. Rosenbaum
528,Beth David Esteban Bravo
529,Ravi Udyawar


In [450]:
# prompt: i want to add data_p2 in to data0 but as i add the size of the data0 would not increase

data_p2 = data_p2[['Series_Title', 'Director', 'Genre', 'Star1', 'Star2', 'Star3', 'Star4']]
data = pd.concat([data, data_p2], ignore_index=True)


In [451]:
data_p2.head()
data_p2.shape

(531, 7)

In [452]:
data.head(1)
data.shape

(1531, 7)

In [453]:
data.isna().sum()

,0
Series_Title,0
Director,4
Genre,7
Star1,22
Star2,55
Star3,70
Star4,76


In [454]:
data = data.dropna(how='any')

In [455]:
data.isna().sum()

,0
Series_Title,0
Director,0
Genre,0
Star1,0
Star2,0
Star3,0
Star4,0


In [456]:
data.shape

(1453, 7)

In [457]:
data['Series_Title'] =data['Series_Title'].str.lower()

In [458]:
data.head()

,Series_Title,Director,Genre,Star1,Star2,Star3,Star4
0,the shawshank redemption,Frank Darabont,Drama,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler
1,the godfather,Francis Ford Coppola,Crime Drama,Marlon Brando,Al Pacino,James Caan,Diane Keaton
2,the dark knight,Christopher Nolan,Action Crime Drama,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
3,the godfather: part ii,Francis Ford Coppola,Crime Drama,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton
4,12 angry men,Sidney Lumet,Crime Drama,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler


In [459]:
data['comb'] = data['Star1']+' '+data['Star2']+' '+data['Star3']+' '+data['Star4']+' '+data['Genre']

In [460]:
data

,Series_Title,Director,Genre,Star1,Star2,Star3,Star4,comb
0,the shawshank redemption,Frank Darabont,Drama,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,Tim Robbins Morgan Freeman Bob Gunton William ...
1,the godfather,Francis Ford Coppola,Crime Drama,Marlon Brando,Al Pacino,James Caan,Diane Keaton,Marlon Brando Al Pacino James Caan Diane Keato...
2,the dark knight,Christopher Nolan,Action Crime Drama,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Christian Bale Heath Ledger Aaron Eckhart Mich...
3,the godfather: part ii,Francis Ford Coppola,Crime Drama,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,Al Pacino Robert De Niro Robert Duvall Diane K...
4,12 angry men,Sidney Lumet,Crime Drama,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,Henry Fonda Lee J. Cobb Martin Balsam John Fie...
...,...,...,...,...,...,...,...,...
1524,the incredible jessica james,Jim Strouse,Romance Comedy,Jessica Williams,Chris O'Dowd,Keith Stanfield,Noël Wells,Jessica Williams Chris O'Dowd Keith Stanfield ...
1525,can't buy my love,Farhad Mann,Romance,Adelaide Kane,Benjamin Hollingsworth,Jean Louisa Kelly,James Remar,Adelaide Kane Benjamin Hollingsworth Jean Loui...
1526,thick lashes of lauri mäntyvaara,Hannaleena Hauru,Romance Comedy,Inka Haapamäki,Rosa Honkonen,Tiitus Rantala,Santeri Helinheimo Mäntylä,Inka Haapamäki Rosa Honkonen Tiitus Rantala Sa...
1527,cop and a half: new recruit,Jonathan A. Rosenbaum,Crime Comedy Action Family,Lou Diamond Phillips,Wallace Shawn,Gina Holden,Lulu Wilson,Lou Diamond Phillips Wallace Shawn Gina Holden...


In [461]:
data.drop_duplicates(subset=['Series_Title'], keep='last', inplace=True)

In [462]:
data

,Series_Title,Director,Genre,Star1,Star2,Star3,Star4,comb
0,the shawshank redemption,Frank Darabont,Drama,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,Tim Robbins Morgan Freeman Bob Gunton William ...
1,the godfather,Francis Ford Coppola,Crime Drama,Marlon Brando,Al Pacino,James Caan,Diane Keaton,Marlon Brando Al Pacino James Caan Diane Keato...
2,the dark knight,Christopher Nolan,Action Crime Drama,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Christian Bale Heath Ledger Aaron Eckhart Mich...
3,the godfather: part ii,Francis Ford Coppola,Crime Drama,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,Al Pacino Robert De Niro Robert Duvall Diane K...
4,12 angry men,Sidney Lumet,Crime Drama,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,Henry Fonda Lee J. Cobb Martin Balsam John Fie...
...,...,...,...,...,...,...,...,...
1524,the incredible jessica james,Jim Strouse,Romance Comedy,Jessica Williams,Chris O'Dowd,Keith Stanfield,Noël Wells,Jessica Williams Chris O'Dowd Keith Stanfield ...
1525,can't buy my love,Farhad Mann,Romance,Adelaide Kane,Benjamin Hollingsworth,Jean Louisa Kelly,James Remar,Adelaide Kane Benjamin Hollingsworth Jean Loui...
1526,thick lashes of lauri mäntyvaara,Hannaleena Hauru,Romance Comedy,Inka Haapamäki,Rosa Honkonen,Tiitus Rantala,Santeri Helinheimo Mäntylä,Inka Haapamäki Rosa Honkonen Tiitus Rantala Sa...
1527,cop and a half: new recruit,Jonathan A. Rosenbaum,Crime Comedy Action Family,Lou Diamond Phillips,Wallace Shawn,Gina Holden,Lulu Wilson,Lou Diamond Phillips Wallace Shawn Gina Holden...


In [463]:
data.to_csv('data_combin.csv',index=False)